In [ ]:
### Calibration
if mach == 0.6:
    dt_ref = 0.0001 * subsampling
    shed_period = 0.03

elif mach == 0.1:
    dt_ref = 0.001 * subsampling
    shed_period = 0.18

elif mach == 0.01:
    dt_ref = 0.02 * subsampling
    shed_period = 1.84

dim_factor = shed_period * 0.16 # strouhal
alpha = 0.2 # calibration parameter

# optimal parameters 
b1_opt = b1
b2_opt = b2
L1_opt = L1
L2_opt = L2
Q_opt = Q_

# computing delta denominator
delta_denominator = 0
delta_denominator += np.sum(np.square(b1))
delta_denominator += np.sum(np.square(b2))
delta_denominator += np.sum(np.square(L1))
delta_denominator += np.sum(np.square(L2))
delta_denominator += np.sum(np.square(Q_))

# computing epsilon denominator
epsilon_denominator = 0
err_zero = 0
t_cal = shed_period * 5
steps = int(t_cal / (dt * dim_factor))
for i in range(steps):
    sample = int((dt * dim_factor)/dt_ref) * i
    a = coeffs[:num,sample]
    a_dot = np.empty_like(a)
    for k in range(a_dot.shape[0]):
        a_dot[k] = (nu * b1[k] + b2[k] + np.inner((nu*L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_[k]),np.expand_dims(a,1)))
    a_opt = (coeffs[:num,sample+1] - coeffs[:num,sample]) / dt_ref
    epsilon_denominator += np.sum(np.square(a_opt - a_dot))
    err_zero += np.sum(np.square(a_opt))

def epsilon():

    global Q_, L1, L2, b1, b2, nu, num
    global Q_opt, L1_opt, L2_opt, b1_opt, b2_opt
    
    # calibration period 
    t_cal = shed_period * 5
    steps = int(t_cal / (dt * dim_factor))
    
    numerator = 0
    
    for i in range(steps):
        sample = int((dt * dim_factor)/dt_ref) * i

        # optimal activation
        a = coeffs[:num,sample]
        # construct hypothesized activation flow
        a_dot_opt = np.empty_like(a)
        for k in range(a_dot.shape[0]):
            a_dot_opt[k] = (nu * b1_opt[k] + b2_opt[k] + np.inner((nu*L1_opt[k,:]+L2_opt[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_opt[k]),np.expand_dims(a,1)))
        
        # optimal activation flow
        a_opt = (coeffs[:num,sample+1] - coeffs[:num,sample]) / dt_ref

        numerator += np.sum(np.square(a_opt - a_dot_opt))

    return numerator / epsilon_denominator

# Delta term
def delta():
    # normalized distance to hypothesized model
    global Q_, L1, L2, b1, b2, nu, num
    global Q_opt, L1_opt, L2_opt, b1_opt, b2_opt

    numerator = 0
    denom = 0

    # manual computation of components
    numerator += np.sum(np.square(b1_opt - b1))
    numerator += np.sum(np.square(b2_opt - b2))
    numerator += np.sum(np.square(L1_opt - L1))
    numerator += np.sum(np.square(L2_opt - L2))
    for i in range(num):
        numerator += np.sum(np.square(Q_opt[i] - Q_[i]))

    return numerator / delta_denominator

# gradient of f (opt_parameters)
def grad():
    global num

    derivative = np.zeros(num)
    for k in range(num):
        derivative[k] += 0

    return 0

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.plot(coeffs[0,:],coeffs[1,:],linewidth=1, linestyle="dashed", label="Mode 0 over Mode 1")
ax.plot(coeffs[2,:],coeffs[3,:],linewidth=1, linestyle="dashed", label="Mode 2 over Mode 3")
#ax.plot(coeffs[4,:],coeffs[5,:],linewidth=1, linestyle="dashed", label="Mode 4 over Mode 5")
ax.set_xlabel("Mode x",size=15)
ax.set_ylabel("Mode y",size=15)
plt.legend()


fig, ax = plt.subplots(1,1,figsize=(8,8))
ax.plot(sol.y[0,-3000:len(t)],sol.y[1,-3000:len(t)],linewidth=1, linestyle="dashed", label="Mode 0 over Mode 1")
ax.plot(sol.y[2,-3000:len(t)],sol.y[3,-3000:len(t)],linewidth=1, linestyle="dashed", label="Mode 2 over Mode 3")
#ax.plot(sol.y[4,-1000:len(t)],sol.y[5,-1000:len(t)],linewidth=1, linestyle="dashed", label="Mode 4 over Mode 5")
ax.set_xlabel("Mode x",size=15)
ax.set_ylabel("Mode y",size=15)
plt.legend()

In [ ]:
# reconstruct flows
rec_len = 100
ssubsampling=20
rec = np.repeat(np.expand_dims(q_avg,1),rec_len,1)
for i in range(num):
    rec += np.outer(pod_modes[:,i],sol.y[i,-100*ssubsampling::ssubsampling])

In [ ]:
diff = np.zeros(200)
for k in tqdm(range(200)):
    nu = 3.0 + k * 0.005 

    sol = solve_ivp(galerkin_system,(t0,tmax),a0,method='LSODA',t_eval=sampling_span)
    
    # Limit cycle -> activation limits
    max_activ = np.empty(num)
    min_activ = np.empty(num)

    for i in range(num):
        max_activ[i] = coeffs[i,:].max()
        min_activ[i] = coeffs[i,:].min()

    # Limit cycle -> activation limits of ROM
    max_activ_rom = np.empty(num)
    min_activ_rom = np.empty(num)

    for i in range(num):
        max_activ_rom[i] = sol.y[i,-int(len(t)/2):].max()
        min_activ_rom[i] = sol.y[i,-int(len(t)/2):].min()

    diff[k] = np.sum(np.divide(abs(max_activ - max_activ_rom) + abs(min_activ - min_activ_rom),1))

In [ ]:

if controlled==True:
    def galerkin_system(t,a):
        # set global variables references
        global Q_, L1, L2, b1, b2, nu
        T = 1 / (0.165 * mach * sos) 
        T1 = T / 2 # based on doubled vortex shedding frequency f=1/T
        T2 = T * 2 
        A = 1
        a_dot = np.empty_like(a)
        gamma = np.sin((2*np.pi*t*0.005)/(T1)+A*np.pi*np.sin((2*np.pi*t*0.005)/(T2)))
        for k in range(a_dot.shape[0]):
            a_dot[k] = (nu * b1[k] + b2[k] + np.inner((nu*L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_[k]),np.expand_dims(a,1))) + gamma * (nu * d1[k] + d2[k] + np.inner(g[k],a) + gamma * f[k]) #+ h[k] * dgamma
        return a_dot

# Limit Cycles

In [ ]:
# Limit cycle -> activation limits
max_activ = np.empty(num)
min_activ = np.empty(num)

for i in range(num):
    max_activ[i] = coeffs[i,:].max()
    min_activ[i] = coeffs[i,:].min()

# Limit cycle -> activation limits of ROM
max_activ_rom = np.empty(num)
min_activ_rom = np.empty(num)

for i in range(num):
    max_activ_rom[i] = sol.y[i,-int(len(sampling_span)/2):].max()
    min_activ_rom[i] = sol.y[i,-int(len(sampling_span)/2):].min()

# Penalty Stabilization

In [ ]:
### bc penalty term
# velocity on boundary 
boundary_weight = np.zeros(3*n)

# compute ds
rad = mesh.points_pol[-200,0]
ds = 2 * np.pi * rad / 199

# set boundary weights for skalar product
boundary_weight[n-200:n] = ds
boundary_weight[2*n-200:2*n] = ds
boundary_weight[3*n-200:3*n] = ds * mach_weight

# state based freestream condition
q_inf = np.mean(q_fluc,1) # time averaged boundary

# bc
G1 = np.empty((num,num))
G2 = np.empty(num)
for i in range(num):
    for j in range(i,num):
        G1[i,j] = skalar_product(pod_modes[:,i],pod_modes[:,j],boundary_weight)
        G1[j,i] = G1[i,j]
    G2[i] = skalar_product(pod_modes[:,i],q_inf,boundary_weight)

In [ ]:
    # doesnt work properly -> unphysical
elif stabilization=="penalty":
        if mach == 0.6:
            # tbd
            tau = 6
        elif mach == 0.1:
            # tbd
            tau = 4
        elif mach == 0.01:
            # tbd
            tau = 8


    elif stabilization=="penalty":
        if mach == 0.6:
            # tbd
            tau = 6 
        elif mach == 0.1:
            # tbd
            tau = 4
        elif mach == 0.01:
            # tbd
            tau = 8

In [ ]:
# Optimization Loop
tmp = 1
cost = 0
counter = 0
maxIter = 10
alpha = 0.001
while(abs(cost) - abs(tmp) < epsilon):
    # max iteration loop break
    if counter > maxIter:
        break
    # Setup AD Taping
    RealReverse.setActive() 
    if counter==0:
        design_vars = np.array([RealReverse(1.0) for _ in range(design_dim)])
    else:
        design_vars =  np.array([design_var for design_var in design_vars])                                              
    # set tape variables
    for i in range(design_dim):                                                 
        design_vars[i].registerInput()

    # copy previous cost
    tmp = cost
    # compute control depending on design variables
    Nt = int(tmax/dt)
    control, control_deriv = gamma(Nt*2,t0,tmax,dt/2,T,design_vars)
    t = t0
    a_tmp = a0
    a_vec = np.zeros((num,Nt))
    for i in range(Nt-2):
        k1 = dt * galerkinSystem(t,a_tmp,control[2*i],control_deriv[2*i])
        k2 = dt * galerkinSystem(t+dt/2,a_tmp+k1/2,control[2*i+1],control_deriv[2*i+1])
        k3 = dt * galerkinSystem(t+dt/2,a_tmp+k2/2,control[2*i+1],control_deriv[2*i+1])
        k4 = dt * galerkinSystem(t+dt,a_tmp+k3,control[2*i+2],control_deriv[2*i+2])
        k = (k1 + 2 * k2 + 2 * k3 + k4) / 6
        a_tmp = a_tmp + k
        a_vec[:,i] = a_tmp
        t = t + dt 
    # compute cost function
    cost = costFunction(a_vec,control[::2])
    # computing gradient with AD
    cost.registerOutput()
    RealReverse.setPassive()
    #cost.setGradient(1.0)
    RealReverse.evaluate()
    for i in range(design_dim):                                                 
        design_vars[i] = design_vars[i] - alpha * design_vars[i].getGradient()

    print(design_vars[7].getGradient())
    # loop output
    cost = 0
    counter += 1

In [ ]:
# parameterized control function
def gamma(Nt,t0,tmax,dt,T,design_vars): 
    # empty control and control deriv arrays
    control = [0.0 for i in range(Nt)]
    control_deriv = [0.0 for i in range(Nt)]
    # loop timesteps
    for i in range(Nt):
        t = t0 + dt * i
        # loop design vars
        for j in range(design_dim): 
            control[i] = control[i] + design_vars[j] * np.sin(j * np.pi * t / (T * 2))
            control_deriv[i] = control_deriv[i] + design_vars[j] * np.cos(j * np.pi * t / (T * 2)) * j * np.pi / (T * 2)
    
        control[i] = control[i] + gamma_start * (1 - t / tmax)
        control_deriv[i] = control_deriv[i] - gamma_start * (1 / tmax)
    
    # return control and control derivative 
    return control, control_deriv 

def costFunction(a,gamma):
    cost = 0
    for i in range(num):
        cost = np.sum(np.sum(np.square(a),0) + np.exp(np.square(gamma) - np.square(gamma_max)) * dt)
    return cost

# Galerkin system
def galerkinSystem(t,a,gamma,dgamma):
    a_dot = np.empty_like(a)
    # iterate dof
    for k in range(a.shape[0]):
        a_dot[k] = nu * b1[k] + b2[k] + np.inner((nu * L1[k,:]+L2[k,:]),a) + np.matmul(np.matmul(np.expand_dims(a,1).T,Q_[k]),np.expand_dims(a,1)) + gamma * (nu * d1[k] + d2[k] + np.inner(g[k],a) + gamma * f[k]) + h[k] * dgamma
    # return activation derivative
    return a_dot

def solver(f,control_function,design_vars,interval,t0,y0,T,dt=dt):
    t0 = interval[0]
    tmax = interval[1]
    Nt = int(tmax/dt)
    control, control_deriv = control_function(Nt*2,t0,tmax,dt/2,T,design_vars)
    t = t0
    y_vec = np.zeros((num,Nt))
    y_vec[:,0] = y0
    for i in range(Nt-1):
        y = y_vec[:,i]
        k1 = dt * f(t,y,control[2*i],control_deriv[2*i])
        k2 = dt * f(t+dt/2,y+k1/2,control[2*i+1],control_deriv[2*i+1])
        k3 = dt * f(t+dt/2,y+k2/2,control[2*i+1],control_deriv[2*i+1])
        k4 = dt * f(t+dt,y+k3,control[2*i+2],control_deriv[2*i+2])
        k = (k1+2*k2+2*k3+k4)/6
        y_vec[:,i+1] = y_vec[:,i] + k
        t = t + dt  
    return y_vec

def costateProblem(t,phi,a,gamma):
    phi_dot = np.zeros_like(phi)
    for k in range(phi.shape[0]):
        A = np.zeros(a.shape[0])
        for i in range(a.shape[0]):
            tmp = 0
            for j in range(a.shape[0]):
                tmp += (Q_[k][i,j] + Q_[k][j,i]) * a[j]
            A[i] = - (nu * L1[k][i] + L2[k][i] + tmp + g[k,i] * gamma)
        phi_dot[k] = np.inner(A,phi) - 0.5 * a[k]
    return phi_dot

def coStateSolver(f,control_function,design_vars,interval,t0,y0,T,dt=dt):
    t0 = interval[0]
    tmax = interval[1]
    Nt = int(tmax/dt)
    control, control_deriv = control_function(Nt*2,t0,tmax,dt/2,T,design_vars)
    t = t0
    y_vec = np.zeros((num,Nt))
    y_vec[:,0] = y0
    for i in range(Nt-1):
        y = y_vec[:,i]
        k1 = dt * f(t,y,control[2*i],control_deriv[2*i])
        k2 = dt * f(t+dt/2,y+k1/2,control[2*i+1],control_deriv[2*i+1])
        k3 = dt * f(t+dt/2,y+k2/2,control[2*i+1],control_deriv[2*i+1])
        k4 = dt * f(t+dt,y+k3,control[2*i+2],control_deriv[2*i+2])
        k = (k1+2*k2+2*k3+k4)/6
        y_vec[:,i+1] = y_vec[:,i] + k
        t = t + dt  
    return y_vec